In [1]:
import requests
import lxml.html as lh
import pandas as pd
import scrapy
import json
import re

import pickle
from bs4 import BeautifulSoup

In [2]:
#looking at scraped data from main webpage
main_url = "https://www.lovelybooks.de/autor/"
result = requests.get(main_url)

result.text[:1000]

'<!DOCTYPE html>\n<html lang="de" xmlns="http://www.w3.org/1999/xhtml" prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml">\n<head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n    <title>Autoren im Gespräch bei LovelyBooks</title>\n    <meta name="description" content="Eine Übersicht über Autoren im Gespräch bei LovelyBooks."/>\n    <meta name="robots" content="index,follow"/>\n    <meta name="baseurl" content="https://www.lovelybooks.de/" />\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n    <meta name="viewport" content="width=device-width">\n    <link rel="canonical" href="https://www.lovelybooks.de/autor/"/>\n    <link rel="icon" type="image/x-icon" href="/favicon.ico"/>\n    <script>\n/*<![CDATA[*/\n\n        function setZoom() {\n            var zoom = window.innerWidth / document.body.getBoundingClientRect().width;\n            document.querySelector(\'#Header\').style.zoom = zoom;\n        }\n        document.addEv

In [6]:
#using Beautiful Soup library to get more structured data
soup = BeautifulSoup(result.text, 'html.parser')

print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="de" prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <title>
   Autoren im Gespräch bei LovelyBooks
  </title>
  <meta content="Eine Übersicht über Autoren im Gespräch bei LovelyBooks." name="description"/>
  <meta content="index,follow" name="robots"/>
  <meta content="https://www.lovelybooks.de/" name="baseurl"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="https://www.lovelybooks.de/autor/" rel="canonical"/>
  <link href="/favicon.ico" rel="icon" type="image/x-icon"/>
  <script>
   /*<![CDATA[*/

        function setZoom() {
            var zoom = window.innerWidth / document.body.getBoundingClientRect().width;
            document.querySelector('#Header').style.zoom = zoom;
        }
        document.addEventListener('DOMContentLoaded', s

In [7]:
#function for using Beautiful Soup
def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

In [8]:
#function to scrap urls of authors on lovelybooks.de
def get_autor_links(link):
    result = requests.get(link)
    soup = BeautifulSoup(result.text, 'html.parser')
    autor_links = soup.find_all(class_='singleItem')
    links=[]
    for link in autor_links:
        links.append(link.a.get("href"))
    return links

In [9]:
#creating a list of url from where we can get the autor urls
seiten_links = []
entry_url='https://www.lovelybooks.de/autor//?seite='
for i in range(857):
    link = entry_url+str(i)
    seiten_links.append(link) 

In [10]:
#getting all author urls in one list
all_autor_links =[]
for seiten in seiten_links:
    all_autor_links.extend(get_autor_links(seiten))

In [11]:
#we have 83582 urls
len(all_autor_links)

83582

In [13]:
#saving the list of autor urls with pickle
with open('autor_links.txt', 'wb') as fp:
    pickle.dump(all_autor_links, fp)

#loading list with pickle
with open ('autor_links.txt', 'rb') as fp:
    all_autor_links = pickle.load(fp)

In [14]:
#adding the main part of the url in the begining
url = 'https://www.lovelybooks.de'
for i in range(len(all_autor_links)):
    all_autor_links[i]= url + all_autor_links[i]

In [16]:
#checking if final urls are ok
all_autor_links[17:27]

['https://www.lovelybooks.de/autor/Martin-Bühler/',
 'https://www.lovelybooks.de/autor/Dietmar-Schlau/',
 'https://www.lovelybooks.de/autor/Stefan-Schweizer./',
 'https://www.lovelybooks.de/autor/E.-L.-Greiff/',
 'https://www.lovelybooks.de/autor/Horst-Eckert/',
 'https://www.lovelybooks.de/autor/Holly--Jane-Rahlens-/',
 'https://www.lovelybooks.de/autor/M.E.-Lee-Jonas/',
 'https://www.lovelybooks.de/autor/Carola-Wegerle/',
 'https://www.lovelybooks.de/autor/Anne-Girard/',
 'https://www.lovelybooks.de/autor/Josh-Malerman/']

In [17]:
#function to scrap particular book urls
def get_book_links(link):
    result = requests.get(link)
    soup = BeautifulSoup(result.text, 'html.parser')
    book_links = soup.find_all(class_='cover')
    links=[]
    for link in book_links:
        links.append(link.a.get("href"))
    return links

In [18]:
#testing the function on signle url
test_link ='https://www.lovelybooks.de/autor/Arantxa-Conrat/'
get_book_links(test_link)

['/autor/Arantxa-Conrat/Wiesnglück-und-Brezelzauber-1188961271-w/',
 '/autor/Arantxa-Conrat/10-Gebote-in-Pink-1137785617-w/',
 '/autor/Arantxa-Conrat/Feindliche-Übernahme-1347516480-w/',
 '/autor/Arantxa-Conrat/Eingecheckt-ins-Liebesglück-1348034310-w/',
 '/autor/Arantxa-Conrat/Finger-Weg-von-den-Männern-1137665538-w/']

In [130]:
#trying to scrap all book urls at once, but killing the servers
#all_book_links =[]
#for link in all_autor_links:
#    all_book_links.extend(get_book_links(link))

ConnectionError: HTTPSConnectionPool(host='www.lovelybooks.de', port=443): Max retries exceeded with url: /autor/Jacqueline-Carey/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000019982281780>: Failed to establish a new connection: [WinError 10060] Próba połączenia nie powiodła się, ponieważ połączona strona nie odpowiedziała poprawnie po ustalonym okresie czasu lub utworzone połączenie nie powiodło się, ponieważ połączony host nie odpowiedział'))

In [136]:
#dividing autor urls in smaller packages
autor_links_1=all_autor_links[0:9999]
autor_links_2=all_autor_links[10000:19999]
autor_links_3=all_autor_links[20000:29999]
autor_links_4=all_autor_links[30000:39999]
autor_links_5=all_autor_links[40000:49999]
autor_links_6=all_autor_links[50000:59999]
autor_links_7=all_autor_links[60000:69999]
autor_links_8=all_autor_links[70000:79999]
autor_links_9=all_autor_links[80000:83580]

In [ ]:
#scraping book urls in packages (1...9) (needs to be repeated 9 time with different file names)
all_book_links_1 =[]
for link in all_autor_links_1:
    all_book_links_1.extend(get_book_links(link))

#and saving the files by pickle
with open('book_links_1.txt', 'wb') as fp:
    pickle.dump(all_book_links_1, fp)

In [11]:
with open ('book_links_1.txt', 'rb') as fp:
    all_book_links_1 = pickle.load(fp)

In [13]:
#getting the book id number from book 
book_id_1 = [i[-13:-3] for i in all_book_links_1]
book_id_1_cleaned = [ x for x in book_id_1 if x.isdigit() ]

In [26]:
#typical url where data is kept and available by json
url = "https://www.lovelybooks.de/mapi/books/103638125/stream?size=1000"

In [14]:
#creating list of book data urls which could be further scrapped
book_data_links=[]
for book_id in book_id_1_cleaned:
    book_data_link = "https://www.lovelybooks.de/mapi/books/{}/stream?size=1000".format(book_id)
    book_data_links.append(book_data_link)

In [67]:
len(book_data_links)

8092

In [79]:
#function to create pandas dataframe from book data url
def create_review_df(link):
    response = requests.get(link).text
    data = json.loads(response)
    if 'content' in data:
        df = pd.DataFrame.from_dict(data['content'])
        if 'bookReview' in df: 
            df = df.dropna(subset=['bookReview'])
            df = df.drop('bookReview', 1).assign(**pd.DataFrame(df['bookReview'].values.tolist()))

            if 'fullTextHtml' in df:
                df = df[['rating', 'fullTextHtml']]
            else:
                df = pd.DataFrame(columns=['rating', 'fullTextHtml'])
        else: 
            df = pd.DataFrame(columns=['rating', 'fullTextHtml'])
    else: 
        df = pd.DataFrame(columns=['rating', 'fullTextHtml'])
    return df

In [ ]:
#function combining scraping and creating pandas dataframe
def scrap_and_create_df(filename, start, stop):
    with open (filename, 'rb') as fp:
        all_book_links = pickle.load(fp)

    book_id = [i[-13:-3] for i in all_book_links]
    book_id_cleaned = [ x for x in book_id if x.isdigit() ]

    book_data_links=[]
    for book_id in book_id_cleaned:
        book_data_link = "https://www.lovelybooks.de/mapi/books/{}/stream?size=1000".format(book_id)
        book_data_links.append(book_data_link)
    
    df_all = pd.DataFrame()
    test_links = book_data_links[start:stop]
    for link in test_links:
        df = create_review_df(link)
        df_all = df_all.append(df)

    return df_all

SCRAPPING each book link one by one with 5000 packages and saving each df in pickle file (needs to be repeated for all book links_.txt (from 1 to 9)

In [88]:
#assigning filename
filename='book_links_6.txt'

In [89]:
#scrapping first 5000 data
df1 = scrap_and_create_df(filename,0,5000)

In [91]:
#scrapping second 5000 data
df2 = scrap_and_create_df(filename,5001,10000)

In [92]:
#scrapping third 5000 data
df3 = scrap_and_create_df(filename,10001,15000)

In [93]:
#scrapping 4th 5000 data
df4 = scrap_and_create_df(filename,15001,20000)

In [94]:
#creating one final df
df_all = pd.DataFrame()
df_all = df_all.append(df1)
df_all = df_all.append(df2)
df_all = df_all.append(df3)
df_all = df_all.append(df4)
df_all.shape

(43178, 2)

In [95]:
#saving df to pickle
df_all.to_pickle("lovelybooks_df_7.pkl")